### 学习目标

- 用Git进行自动化脚本的版本管理
- 用Git保证团队报告自动化脚本的版本同步

### 知识基础

- Git基础知识

### 1. 文档结构调整

在多人协作时，我们要尽保持模板的同步，同时每个人对模板的完善都能集合在一起，而具体工作的内容不应该产生干扰。所以对于变动频繁的文件，我们可以集合起来然后进行同步时忽略处理。
对于第8章的文档结构，会常发生变动的有：
- ***report*** 自动化报告的工作目录
    - main.ipynb 文件的主执行和调试文件，每次执行都会变化
    - ***data***
        - *dynamic_data.csv*
        - *static_data.csv*
    - ***image*** 存放生成的图片文件
    - ***report*** 存放生成的报告
    - **configs.py** 设置报告的参数，比如作者，日期，数据源等等，每次使用时可以仅仅修改设置便生成不同报告
    
而这些我们可以打包在一起放入***report***文件夹，把文档结构调整为：
- ***report*** 自动化报告的工作目录
    - ... 其他文件不动
    - **configs.py** 在此需要添加设置报告的编号，且报告编号需要唯一
    - main.ipynb 文件的主执行和调试文件，不会执行，仅仅作为母版
    - ***data***
        - *static_data.csv*
    - ***reports*** 存放生成的报告
        - ***report1*** 报告1
        - ***report2*** 报告2
            - main.ipynb 从上上级文件夹复制，每次执行时都会变化
            - **configs.py** 基于上上级文件夹中的**configs.py**文件生成，作为此报告设置的记录
            - ***data***
                - *dynamic_data.csv*
            - ***image*** 存放生成的图片文件

**注意：**

在文档结构调整后，我们还需要添加辅助用的小脚本，用途为：
1. 基于**configs.py**在***reports***文件夹生成制作报告用的文件夹，并生成如下的目录结构
    - ***report1*** 报告1
        - main.ipynb  从上上级文件夹复制，每次执行时都会变化
        - **configs.py** 基于上上级文件夹中的**configs.py**文件生成，作为此报告设置的记录
        - ***data***
        - ***image*** 存放生成的图片文件
2. 检查***reports***文件夹中的目录结构是否正确

**检查目录的小例子：**

更复杂和完善的请根据报告形式自行编写

In [1]:
import os
import logging
logging.info("当前目录 : {}".format(os.getcwd()))

In [3]:
def check_folder(folder_name):
    if os.path.exists(folder_name):
        logging.info("文件夹已创立：{}".format(folder_name))
    else:
        logging.warning("文件夹未创立：{}".format(folder_name))
        os.makedirs(folder_name)
        logging.info("文件夹创立完毕：{}".format(folder_name))

In [2]:
check_folder("./Example_2")
# 修改当前工作目录
os.chdir("./Example_2")
logging.info("当前目录 : {}".format(os.getcwd()))

In [4]:
import configs
check_folder(configs.report_name)

In [5]:
# 创建文件夹
for folder in ['data', 'image']:
    check_folder(r"./{}/{}".format(configs.report_name, folder))

In [9]:
# 复制文件
import shutil
for filename in ['main.ipynb', 'configs.py']:  
    if os.path.exists(r"./{}/{}".format(configs.report_name, filename)):
        logging.warning("文件未添加：./{}/{}".format(configs.report_name, filename))
    else:
        shutil.copy(filename, r"./{}/{}".format(configs.report_name, filename))
        logging.info("已复制文件：{}".format(filename))

### 2. main.ipynb的拆分
这里分为两种情况：
1. 报告尚未完全自动化，还需要人工干涉
由于main.ipynb在每次执行时结果会不同，变动时无法保持同步，所以可以对其内容进行拆分，分为方法部分和人工设置部分，以实现方法部分通用，人工设置部分每篇报告都输入不同。
2. 报告已经完全自动化
这时候main.ipynb仅仅是作为一个调试工具，把其中的方法保存为main.py，之后直接执行main.py即可



**TODO :**
具体拆分方法和main.ipynb的组织方式会在之后补充。

### 3. 设置git添加时的忽略文件

所以对于生成报告所用的公用文件，我们用Git进行版本控制，而每次报告都会不同的设置或者数据，以及生成文件，我们需要让Git忽略掉它们，但是还要保证文件的组成结构没有变化。

比如对于调整前的文档结构，.gitignore的文件内容添加如下
```
\__pycache__
*.pyc
.ipynb_checkpoints
data/data.csv
image/*
report/*
```
分别忽略了：
- python导入模块时的生成文件 `\__pycache__、*.pyc`
- ipynb的历史记录文件 `.ipynb_checkpoints`
- 每份报告都会

而对于调整后的文档结构，.gitignore的文件内容可改为添加如下
```
\__pycache__
*.pyc
.ipynb_checkpoints
report/*
```